**Лабораторная работа №5**  
**Метрики качества классификации**  
Выполнила: Иппо Вера, гр. P4117  
**Цель работы:** рассмотреть различные метрики качества классификации, входящих
в состав библиотеки scikit-learn.  
**Исходные данные**   
Датасет: https://archive.ics.uci.edu/ml/datasets/Letter+Recognition  
Предметная область: буквы латинского алфавита  
Задача: определить, какой из букв латинского алфавита соответствует набор характеристик ее написания.  
Количество записей: 20000  
Количество атрибутов: 16  
**Атрибуты:**  
lettr capital letter (26 values from A to Z)  
x-box horizontal position of box (integer)  
y-box vertical position of box (integer)  
width width of box (integer)  
high height of box (integer)  
onpix total # on pixels (integer)  
x-bar mean x of on pixels in box (integer)  
y-bar mean y of on pixels in box (integer)  
x2bar mean x variance (integer)  
y2bar mean y variance (integer)  
xybar mean x y correlation (integer)  
x2ybr mean of x x y (integer)  
xy2br mean of x y y (integer)  
x-ege mean edge count left to right (integer)  
xegvy correlation of x-ege with y (integer)  
y-ege mean edge count bottom to top (integer)  
yegvx correlation of y-ege with x (integer)  

Расчет характеристик будет проводиться для метода опорных векторов и Random Forest алгоритма.

Точность классификации (Classification Accuracy)

In [108]:
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

In [8]:
#загрузка датасета
def load_dataset(filename):
	csv_dataset = pandas.read_csv(filename, header=None).values
	dataset=csv_dataset
	return dataset
	
#разбиение на обучающую и тестовую выборки
dataset=load_dataset("letter-recognition.csv")
letter_attr = dataset[:,1:] # список атрибутов (признаков) для каждой буквы
letter_class = dataset[:,0] # классы букв
data_train, data_test, class_train, class_test = train_test_split(letter_attr, letter_class, test_size=0.3)

In [7]:
def calc_classification_accuracy(method_name, method):
    scoring = 'accuracy'
    accuracy = cross_val_score(method, letter_attr, letter_class, cv=5,scoring=scoring)
    print("Accuracy for ",method_name,": %.3f (%.3f)" % (accuracy.mean(),accuracy.std()))

In [9]:
forest = RandomForestClassifier(n_estimators=100).fit(data_train, class_train)
svc = svm.SVC(kernel='rbf', gamma=0.04, C=10,probability=True).fit(data_train, class_train)

In [20]:
calc_classification_accuracy("Random Forest",forest)
calc_classification_accuracy("SVC",svc)

Accuracy for  Random Forest : 0.966 (0.005)
Accuracy for  SVC : 0.981 (0.006)


Логарифм функции правдоподобия (Logarithmic Loss)

In [94]:
def calc_logarithmic_loss(method_name, method):
    scoring = 'neg_log_loss'
    methodLoss = cross_val_score(method, letter_attr, letter_class, cv=5, scoring=scoring)
    print("Logarithmic loss for ",method_name,": %.3f (%.3f)" % (methodLoss.mean(), methodLoss.std()))

In [ ]:
calc_logarithmic_loss("Random Forest",forest)
calc_logarithmic_loss("SVC",svc)

Logarithmic loss for  Random Forest : -0.276 (0.009)  
Logarithmic loss for  SVC : -0.088 (0.003)

Область под кривой ошибок (Area Under ROC Curve)

In [124]:
def calc_auc(method, method_name):
    new_dataset=numpy.array([element for element in dataset if (element[0] in ['O','I'])])
    x_attr=new_dataset[:, 1:]
    y_class=new_dataset[:, 0]
    bin_y= label_binarize(list(y_class), classes=['A', 'O'])
    y_class=numpy.array([element[0] for element in bin_y ])

    x_train, x_test, y_train, y_test = train_test_split(x_attr, y_class, test_size=0.3)
    
    classifier = OneVsRestClassifier(method)
    method_result = classifier.fit(x_train, y_train)
    scoring = 'roc_auc'
    auc = cross_val_score(method_result, x_attr, y_class, cv=5, scoring=scoring)
    print("auc ",method_name,": %.3f (%.3f)" % (auc.mean(), auc.std()))

In [125]:
new_forest = RandomForestClassifier(n_estimators=100)
new_svc = svm.SVC(kernel='rbf', gamma=0.04, C=10,probability=True)

In [126]:
calc_auc(new_forest,"Random forest")
calc_auc(new_svc,"SVC")

auc  Random forest : 1.000 (0.000)
auc  SVC : 1.000 (0.000)


Матрица неточностей (Confusion Matrix)

In [15]:
def calc_confusion_matrix(method_name, method):
    predict = method.predict(data_test)
    matrix = confusion_matrix(class_test, predict)
    print("Confusion matrix ",method_name,":\n", matrix)

In [54]:
calc_confusion_matrix("Random Forest",forest)
calc_confusion_matrix("SVC",svc)

Confusion matrix  Random Forest :
 [[242   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   1   0]
 [  0 229   0   1   1   0   0   3   0   0   0   0   0   0   0   0   0   1
    0   0   1   4   0   0   0   0]
 [  0   0 224   0   4   1   2   0   0   0   0   0   0   0   1   0   3   0
    0   0   0   0   0   0   0   0]
 [  1   1   0 246   0   0   0   2   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0]
 [  0   1   0   0 192   0   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   1]
 [  0   3   0   1   1 218   0   0   0   0   0   0   1   1   0   6   0   0
    1   7   0   0   1   0   1   0]
 [  0   1   3   4   2   0 225   0   0   0   0   0   0   0   1   0   3   1
    0   0   0   0   0   0   0   0]
 [  0   1   0   1   0   2   3 210   0   0   7   0   0   0   0   0   1   4
    0   0   0   0   0   1   0   0]
 [  0   2   0   0   0   1   0   0 223   9   0   0   0   0   0   4   0   0
    1   0   0   0  

Отчет классификации (Classification Report)

In [13]:
def create_classification_report(method_name, method):
    predict = method.predict(data_test)
    report = classification_report(class_test, predict)
    print("Classification_report ",method_name,":\n", report)

In [14]:
create_classification_report("Random Forest",forest)
create_classification_report("SVC",svc)

Classification_report  Random Forest :
              precision    recall  f1-score   support

          A       0.99      0.99      0.99       244
          B       0.92      0.95      0.93       240
          C       0.98      0.95      0.97       235
          D       0.96      0.98      0.97       250
          E       0.91      0.98      0.95       195
          F       0.94      0.90      0.92       241
          G       0.96      0.94      0.95       240
          H       0.93      0.91      0.92       230
          I       0.98      0.93      0.96       240
          J       0.95      0.97      0.96       205
          K       0.93      0.90      0.92       220
          L       0.99      0.97      0.98       243
          M       0.98      0.99      0.98       244
          N       0.99      0.95      0.97       243
          O       0.97      0.97      0.97       230
          P       0.95      0.97      0.96       243
          Q       0.96      0.98      0.97       220
     

**Выводы:**
В результате исследования, проведенного в ходе данной лабораторной работы, можно сделать вывод, что SVC алгоритм по всем метрикам лучше, чем алгоритм Random Forest. Оба метода классификации подходят для обработки данного датасета и показывают очень хорошие результаты.Также интересно отметить, что параметры precision и recall алгоритма SVC для  некоторых букв достигает 1, что значит для них не было ни ложных срабатываний, ни ложных пропусков. Возможно это связано с полнотой и большим размером датасета. К сожалению, в связи с большим количеством классов не удалось правильно рассчитать площадь области под кривой ошибок, так как выборка по двум буквам мала и результат не сравним с остальными метриками.